# an example about loading data from tfrecord

trying to use the example from 
[http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/)

In [1]:
%pylab inline
plt.style.use('ggplot')
import tensorflow as tf
import pandas as pd
IMAGE_HEIGHT = 200
IMAGE_WIDTH = 320
BATCH_SIZE=3

Populating the interactive namespace from numpy and matplotlib


In [2]:
val_filename = '/notebooks/Minecraft-AI/mc-data/mesa_3_vs_forest_1_test.tfrecords'

In [3]:
labels ={"mesa":0, "forest":1}
tfrecords_filename = '/notebooks/Minecraft-AI/mc-data/mesa_3_vs_forest_1_train.tfrecords'


In [8]:
def read_and_decode(filename_queue):
    
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string),
        'mask_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
        })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    annotation = tf.decode_raw(features['mask_raw'], tf.uint8)
    label =  tf.cast(features['label'], tf.int32)
    
    height = tf.cast(features['height'], tf.int32)
    width = tf.cast(features['width'], tf.int32)
    
    #tf.stack equal to tf.stack([x, y, z]) = np.asarray([x, y, z])
    image_shape = tf.stack([height, width, 3])
    annotation_shape = tf.stack([height, width, 1])
    
    image = tf.reshape(image, image_shape)
    annotation = tf.reshape(annotation, annotation_shape)
    
    image_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=tf.int32)
    annotation_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=tf.int32)
    
    # Random transformations can be put here: right before you crop images
    # to predefined size. To get more information look at the stackoverflow
    # question linked above.
    
    resized_image = tf.image.resize_image_with_crop_or_pad(image=image,
                                           target_height=IMAGE_HEIGHT,
                                           target_width=IMAGE_WIDTH)
    
    resized_annotation = tf.image.resize_image_with_crop_or_pad(image=annotation,
                                           target_height=IMAGE_HEIGHT,
                                           target_width=IMAGE_WIDTH)
    
    
    images, annotations,label = tf.train.shuffle_batch( [resized_image, resized_annotation,label],
                                                 batch_size=10,
                                                 capacity=30,
                                                 num_threads=2,
                                                 min_after_dequeue=10)
    
    return images, annotations, label, width, height

In [5]:
print filename_queue.size()

NameError: name 'filename_queue' is not defined

In [6]:

def scaleImg(img):
    scaled_img = img.astype(numpy.float32)
    return scaled_img /255

In [11]:
label

NameError: name 'label' is not defined

In [9]:
#this is for validation data
filename_queue = tf.train.string_input_producer(
    [tfrecords_filename])

# Even when reading in multiple threads, share the filename
# queue.
image, annotation, label, w, h = read_and_decode(filename_queue)

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session()  as sess:
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # Let's read off 3 batches just for example
    for i in xrange(3):
        img, anno, lab = sess.run([image, annotation, label])
        print('current batch')
        
        # We selected the batch size of two
        # So we should get two image pairs in each batch
        # Let's make sure it is random
        fig = plt.figure(figsize=(10,10))
        fig.add_subplot(121)
#         plt.title("label:{}".format(labels[lab[0]]))
        print(lab)
        plt.imshow(scaleImg(img[0,:,:,:]))
        fig.add_subplot(122)
        plt.imshow(scaleImg(anno[0, :, :, 0]),cmap=plt.cm.Greys)
        
        fig = plt.figure(figsize=(10,10))
        fig.add_subplot(121)
#         plt.title("label:{}".format(labels[lab[1]]))
        plt.imshow(scaleImg(img[1,:,:,:]))

        fig.add_subplot(122)
        plt.imshow(scaleImg(anno[1, :, :, 0]),cmap=plt.cm.Greys)
    
    coord.request_stop()
    coord.join(threads)

ValueError: All shapes must be fully defined: [TensorShape([Dimension(200), Dimension(320), Dimension(3)]), TensorShape([Dimension(200), Dimension(320), Dimension(1)]), TensorShape([Dimension(None)])]